# Requirements
Basic knowledge of Python is needed to follow this notebook. Check the [subjects listed in this course](https://www.kaggle.com/learn/python).

# Load packages
First we are going to install the [`ifcopenshell`](http://ifcopenshell.org/) package. *IfcOpenShell* is an open source software library that helps users and software developers to work with the IFC file format. The IFC file format can be used to describe building and construction data. The format is commonly used for Building Information Modelling (BIM).<br>
<br>
Run the following code to install the package in the curren environment:

In [ ]:
conda install -c conda-forge -c oce -c dlr-sc -c ifcopenshell ifcopenshell

And now we import the packages we are going to use in this notebook:

In [ ]:
import ifcopenshell

# Intro to some key functions in `ifcopenshell` and IFC documentation

*This Notebook was originally published in [this repository](https://github.com/bimfag/intro-python-bim), please check it out to find more information about ifcOpenShell for python.*

Ifcopenshell is a library that can be used to parse and handle IFC model data. It works in both C++ and python, for both Ifc2x3 and IFC4. For a collection of information on IFC please refer to the [buildingSMART IFC pages](https://technical.buildingsmart.org/standards/ifc/). 

It might also be usefull to read up on the [Implementation Guides](https://technical.buildingsmart.org/resources/ifcimplementationguidance/). The [ifcopenshell academy](http://academy.ifcopenshell.org/) is a good resources as well. 

One particular supportive tutorial for this notebook is the [Using The Parsing Functionality of Ifcopenshell Interactively Tutorial](http://academy.ifcopenshell.org/using-the-parsing-functionality-of-ifcopenshell-interactively/).

The topics for this notebook are: 

- [Opening an .ifc file and create a file object](#Opening-an-.ifc-file-and-create-a-file-object)
- [The file.by_type function](#The-file.by_type-function)
- [The is_a function](#The-is_a-function)
- [The "." operator](#The-"."-operator)
- [Using the documentation](#Using-the-documentation)
- [Extra reading](#Extra-reading)

## Opening an .ifc file and create a file object

With ifcopenshell you could start interacting with a file through the `open()` function. The only argument to this function is a path to the `.ifc` file:

In [ ]:
f = ifcopenshell.open("../input/example-ifc-file/Grethes-hus-bok-2.ifc")

## The file.by_type function

In ifcopenshell the function `by_type()` can be applied on the file object. The argument for this function is an Ifc type. Like, eg.`IfcWall` or other ifc types in the schema. If the entity is in the file, the function will return a tuple listing up all the entities of that type in the file. If it is not in the file, the function returns an empty tuple. 

Since we have loaded an architectural model, entities such as `IfcWall`, `IfcWindow`, or `IfcDoor` are probable to be in the model; however `IfcFlowSegment` is not likely to be in this model. 

This types are called *entities* and you can check all of them [here](https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD2_TC1/HTML/link/alphabeticalorder-entities.htm).

In [ ]:
## Here we use the by_type() function to test on the entities above
no_walls = len(f.by_type("IfcWall"))
no_windows = len(f.by_type("IfcWindow"))
no_doors = len(f.by_type("IfcDoor"))
no_flo_segments = len(f.by_type("IfcFlowSegment"))

print("There are {} walls, {} windows, {} doors and {} flow segments in the file".format(no_walls,no_windows,no_doors,no_flo_segments))

## The is_a function

Another usefull function in `ifcopenshell` is the `ifc_object.is_a()` that returns the IFC type of the `ifc_object`. It could also be used as 
```
ifc_object.is_a("particular ifc type")
```
to check if the `ifc_object` you have is of a particular ifc type. If it is of the particular type, it returns `True` and `False` if its not. 

In [ ]:
# get the list of all object in file "f" of type IfcWall
ifc_wall_objects = f.by_type("IfcWall")
# get the first wall object in the list
a_wall = ifc_wall_objects[0]
# print the ifc type of "a_wall"
print("a_wall is of type %s"%(a_wall.is_a()))
# Check if "a_wall" is of type IfcWall
print("\nIt is %s that a_wall is of type IfcWall"%(a_wall.is_a("IfcWall")))

## The "." operator

When you have an ifc object you can use the  `.` operator to access its attributes: 
```
ifc_object.Attribute
```
`Name` is a common Attribute of objects as well as `Description`.  


In [ ]:
# print "a_wall"s Name attribute
print("The Name of a_wall is %s"%(a_wall.Name))
# print "a_wall"s Description attribute
print("The Description of a_wall is %s"%(a_wall.Description))

## Using the documentation

### The by_type function

You might wonder what happens if you pass an entity to `by_type()` that is not defined in IFC schema. Then the **function returns an error**. This is because the `ifcopenshell` knows the IFC schema. Hence, when working with `ifcopenshell` [the documentation on the IFC schema defining the model you are working with](https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD2_TC1/HTML/schema/ifcsharedbldgelements/lexical/ifcwall.htm) is very usefull. 

### The is_a function

It is quite obvious that 
```
wall_in_list_of_walls.is_a("IfcWall")
``` 
would return `True` and 
``` 
wall_in_list_of_walls.is_a("IfcWindow")
```
would return `False`. However, did you notice that the type was `IfcWallStandardCase` but also of type `IfcWall`? What if you try:
```
wall_in_list_of_walls.is_a("IfcBuildingElement")
``` 

Always Check [the documentation](https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD2_TC1/HTML/schema/ifcsharedbldgelements/lexical/ifcwall.htm) of the entities you are using and evaluate its inherritance graph.  

In [ ]:
# Try with "IfcBuildingElement", "IfcElement", "IfcProduct" and "IfcRoot":
a_wall.is_a("IfcBuildingElement")

### The "." operator

When you have a reference to a IFC type that is in your file using the `f.by_type()` function, you could refer to the documentation again to see what you could do with it. 

`ifcopenshell` supports the schema, and enables you to access the objects attributes through the `.` operator.

So looking into [the documentation of IfcWall](https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD2_TC1/HTML/schema/ifcsharedbldgelements/lexical/ifcwall.htm) again, we see that `IfcWall` is inherriting from `IfcRoot`, which gives the following attributes: 

* GlobalId 
* OwnerHistory
* Name
* Description 

In [ ]:
# If our wall is inheriting from IfcRoot, it should have the following attributes:
guid = a_wall.GlobalId
owner_history = a_wall.OwnerHistory
name = a_wall.Name
description = a_wall.Description 

print("GlobalId: {},\nOwnerHistory: {},\nName: {},\nDescription: {}".format(guid,owner_history,name,description))

As showed above the `.` operator works directly on the schema attributes names. As described by the inherritance graph of the `IfcOwnerHistory` objects, `OwningUser` and `OwningApplication` are attributes for this object. These are of the type `IfcPersonAndOrganization` and `IfcApplication` respectively. 

In [ ]:
## OwningUser:
owning_user = owner_history.OwningUser
# attributes inherited from IfcPersonAndOrganization
the_person = owning_user.ThePerson
the_org = owning_user.TheOrganization
roles = owning_user.Roles 
print("Person: {}, \nOrganization: {}, \nRoles: {}\n".format(the_person,the_org,roles))

## OwningApplication: 
owning_app = owner_history.OwningApplication
# attributes inherited from IfcApplication
app_dev = owning_app.ApplicationDeveloper
version = owning_app.Version
app_f_name = owning_app.ApplicationFullName
app_id = owning_app.ApplicationIdentifier

print("Developer: {},\nVersion: {},\nApp Name: {}, \nApp id: {}".format(app_dev,version,app_f_name,app_id))

# Extra reading
- If you want to learn more about `ifcopenshell` and how to use it please check out the [complete course](https://github.com/bimfag/intro-python-bim).
- Get familiar with the IFC documentation, you can find the schema specification (based on the IFC version you are using) [here](https://technical.buildingsmart.org/standards/ifc/ifc-schema-specifications/).
- [IFC Implementation Guides](https://technical.buildingsmart.org/resources/ifcimplementationguidance/)
- [IfcOpenShell Academy](http://academy.ifcopenshell.org/)
- ["Using the parsing functionality of IfcOpenShell interactively" Tutorial](http://academy.ifcopenshell.org/using-the-parsing-functionality-of-ifcopenshell-interactively/)